## Setup

To be able to access all resources within this notebook, make sure Jupyter is launched at the root of this repo. The path of the notebook should be `/notebook/<repo_root>`.

In [5]:
# (Optional) Increase notebook width for all embedded cells to display properly
from IPython.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
# Useful imports
import os
from pathlib import Path
import tempfile

import hydra

# Training an ML planner <a name="training"></a>

## Prepare the training config

In [7]:
# Location of path with all training configs
CONFIG_PATH = '../nuplan/planning/script/config/training'
CONFIG_NAME = 'default_training'

# Create a temporary directory to store the cache and experiment artifacts
SAVE_DIR = Path(tempfile.gettempdir()) / 'tutorial_nuplan_framework'  # optionally replace with persistent dir #NOTES /tmp/tutorial_nuplan_framework
EXPERIMENT = 'training_vector_experiment'#'training_raster_experiment'
LOG_DIR = str(SAVE_DIR / EXPERIMENT)

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={str(SAVE_DIR)}', #NOTES: default_experiment 
    f'cache.cache_path={str(SAVE_DIR)}/cache', #NOTES: default_training
    f'experiment_name={EXPERIMENT}', #NOTES: default_experiment
    'py_func=train', #NOTES: default_training
    '+training=training_vector_model',  # raster model that consumes ego, agents and map raster layers and regresses the ego's trajectory #NOTES:nuplan/planning/script/experiments/training
    'scenario_builder=nuplan_mini',  # use nuplan mini database #NOTES:default_common
    'scenario_filter.limit_total_scenarios=500',  # Choose 500 scenarios to train with #NOTES:default_common
    'lightning.trainer.params.accelerator=ddp_spawn',  # ddp is not allowed in interactive environment, using ddp_spawn instead - this can bottleneck the data pipeline, it is recommended to run training outside the notebook #NOTES:nuplan/planning/script/config/training/lightning/default_lightning.yaml
    'lightning.trainer.params.max_epochs=10',
    'data_loader.params.batch_size=8', #NOTES:nuplan/planning/script/config/training/data_loader/default_data_loader.yaml
    'data_loader.params.num_workers=8',
    # 'splitter=nuplan',
])


## Launch tensorboard for visualizing training artifacts

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

## Launch training (within the notebook)

In [ ]:
from nuplan.planning.script.run_training import main as main_train

# Run the training loop, optionally inspect training artifacts through tensorboard (above cell)
main_train(cfg)

## Launch training (command line - alternative)

A training experiment with the above same parameters can be launched alternatively with:
```
$ python nuplan/planning/script/run_training.py \
    experiment_name=raster_experiment \
    py_func=train \
    +training=training_raster_model \
    scenario_builder=nuplan_mini \
    scenario_filter.limit_total_scenarios=500 \
    lightning.trainer.params.max_epochs=10 \
    data_loader.params.batch_size=8 \
    data_loader.params.num_workers=8
```